# Install KerasHub

In [5]:
!pip install -q tensorflow keras git+https://github.com/keras-team/keras-hub.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Imports and Utilities

In [6]:
import os
os.environ['KERAS_BACKEND'] = 'jax'

import tensorflow as tf
import keras
import keras_hub
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from keras.applications.imagenet_utils import decode_predictions

# EASY: Classify a Single Image with a Pretrained Model

In [7]:
def easy_example():
    classifier = keras_hub.models.ImageClassifier.from_preset("efficientnet_b0_imagenet")

    image_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
    dataset_dir = tf.keras.utils.get_file(origin=image_url, untar=True)
    sample_image_path = f"{dataset_dir}/roses/12240303_80d87f77a3_n.jpg"

    img = keras.utils.load_img(sample_image_path, target_size=(224, 224))
    img_array = np.expand_dims(np.array(img), axis=0)

    preds = classifier.predict(img_array)
    decoded = decode_predictions(preds, top=3)
    print("Easy Example Predictions:", decoded)

# INTERMEDIATE: Train on TFDS rock_paper_scissors

In [ ]:
def intermediate_example():
    dataset, info = tfds.load("rock_paper_scissors", with_info=True, as_supervised=True)
    num_classes = info.features["label"].num_classes

    def preprocess(x, y):
        x = tf.image.resize(x, (224, 224)) / 255.0
        return x, tf.one_hot(y, num_classes)

    train_data = dataset["train"].map(preprocess).batch(32)

    model = keras_hub.models.ImageClassifier.from_preset("mobilenet_v3_small_imagenet", num_classes=num_classes)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    model.fit(train_data, epochs=3)

# ADVANCED: Fine-tune on caltech_birds2011 with Augmentations

In [ ]:
def advanced_example():
    train_data, info = tfds.load("caltech_birds2011", split="train", with_info=True, as_supervised=True)
    num_classes = info.features["label"].num_classes

    def preprocess(x, y):
        x = tf.image.resize(x, (224, 224)) / 255.0
        x = tf.image.random_flip_left_right(x)
        return x, tf.one_hot(y, num_classes)

    train_data = train_data.map(preprocess).shuffle(512).batch(32)

    model = keras_hub.models.ImageClassifier.from_preset("resnet_v1_50_imagenet", num_classes=num_classes)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    model.fit(train_data, epochs=2)

# EXPERT: Backbone + Custom Head on food101

In [ ]:
def expert_example():
    train_data = tfds.load("food101", split="train[:5%]", as_supervised=True)
    test_data = tfds.load("food101", split="validation[:5%]", as_supervised=True)
    NUM_CLASSES = 101

    def preprocess(x, y):
        x = tf.image.resize(x, (224, 224)) / 255.0
        return x, tf.one_hot(y, NUM_CLASSES)

    train_data = train_data.map(preprocess).batch(32)
    test_data = test_data.map(preprocess).batch(32)

    backbone = keras_hub.models.ResNetBackbone.from_preset("resnet_v1_18")
    model = keras.Sequential([
        backbone,
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(NUM_CLASSES, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    model.fit(train_data, epochs=2, validation_data=test_data)

# Run all examples


In [ ]:
easy_example()
intermediate_example()
advanced_example()
expert_example()